In [1]:
import os
import numpy as np
import open3d as o3d
import igl
import meshplot as mp
from matplotlib import pyplot as plt
from FeatureLines import get_mesh, to_pseudo_PLY2, ReadCrestLine
from Display import mesh_lines, Colors
from Deform import Handles
from Flatten import write_TLC_input, read_TLC_result
# from Transfer import crestline_mover
from numpy.typing import NDArray, ArrayLike

In [4]:
"""Optional: Get mesh directly from point cloud
In the experiment, we manually cropped out a sub-section of the mesh 
produced by the code in this cell using Meshlab"""
import numpy as np
name = '' # name of point cloud
pc = o3d.io.read_point_cloud(f"{name}.ply")
o3d.geometry.KDTreeSearchParam.KNNSearch = 10
pc.estimate_normals()
pc.orient_normals_to_align_with_direction(
    orientation_reference=np.asarray([0.0, 0.0, 1.0]))
depth = 10
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
     pc, depth=depth)
remove_vertices_mask = densities < np.quantile(densities, 0.005)
mesh.remove_vertices_by_mask(remove_vertices_mask)
o3d.io.write_triangle_mesh(f"{name}_mesh_d={depth}.ply", mesh)

In [2]:
"""Setup for mesh processing"""
neighbor = 4
# 'depth' here is for record-keeping. Its value does not affect the calculations in this file
depth = 10
mesh_name = f'' # name of mesh
here = "" # full path/file address to this folder
mesh_file = f"{mesh_name}.ply"
ply2_name = f"{mesh_name}_n={neighbor}_ply2.txt"
project_name = f'{mesh_name}_n={neighbor}_depth={depth}'

In [ ]:
"""Prepare input to CrestCODE"""
os.chdir(here)
V, F = get_mesh(mesh_file)
print(f"3D Lidar data: \t{mesh_file}")
print(f"neighbor value: d = {neighbor}")
print(f"PLY2 file: \t{ply2_name}")
to_pseudo_PLY2(name=ply2_name, verts=V, faces=F, neighbor=neighbor)
crestcode_input = f"./setCurvature {ply2_name} output.txt"
print(f"in CrestCODE, enter: {crestcode_input}") # this is done automatically
keeper = "ravines.txt"
# in following line, replace 'where_is_crestcode' with full path to the folder containing CrestCODE code
os.rename(ply2_name, f'where_is_crestcode/{ply2_name}')

In [ ]:
"""Running CrestCODE"""
os.chdir('') # address to CrestCODE installation
os.system('pwd')
os.system(crestcode_input)

In [5]:
"""Collecting CrestCODE results and putting them back to this folder"""
# os.system('rm ridges.txt')
os.system(f'rm {ply2_name}')
# os.system('rm output.txt')
os.system('pwd')
# in the following lines, where_am_i is the full path/file address to this folder
os.rename('ravines.txt', f'where_am_i/{mesh_name}_n={neighbor}_ravines.txt')
os.rename('ridges.txt', f'where_am_i/{mesh_name}_n={neighbor}_ridges.txt')
os.chdir(here)
os.system('pwd')
crestline_V_3d, crestline_E = ReadCrestLine(f"{mesh_name}_n={neighbor}_ravines.txt")

/Users/kronos.di.vlad/Desktop/ComputerGraphics_2023SP/CrestCODE
/Users/kronos.di.vlad/Desktop/ComputerGraphics_2023SP/Open3D_Test


In [33]:
"""Visualizing resulting mesh and crest lines"""
from Display import Colors
import meshplot as mp
PALETTE = Colors()
plot_3d = mp.plot(v=V, f=F, c=PALETTE.GREY)
plot_3d.add_lines(beginning=crestline_V_3d[crestline_E[:,0]], 
                  ending=crestline_V_3d[crestline_E[:,1]], 
                  shading={"line_color": "red", "line_width": 10000.0})
plot_3d.add_points(points=crestline_V_3d, c='red', shading={"point size": 100})

In [ ]:
"""ARAP deform to prepare for SEA algorithm, displaying before-and-after"""
PALETTE = Colors()
handle_indices = [19841, 12438, 26200, 31800, 33350, 78265, 81096, 21838, 68046, 27896, 58194, 11679, 44274, 21332, 97337, 78209]
handle_locations = V[handle_indices]
target_locations = [[1.2001, 1.7609, 0], [1.2602, 1.9212, 0], [1.126, 1.5211, 0], [1.5027, 1.5738, 0], [1.7278, 1.5671, 0], [2.1373, 1.7553, 0], [2.2855, 1.594, 0], [1.3301, 2.1361, 0], [1.9483, 2.155, 0], [1.4107, 2.4159, 0], [1.9082, 2.3955, 0], [1.7145, 2.6293, 0], [0.5692, 1.5454, 0], [0.7124, 1.6473, 0], [2.143, 1.0793, 0], [1.7947, 1.046, 0]]
handle_colors = np.asarray([
    PALETTE.BLUE,   PALETTE.BLUE,   PALETTE.GREEN,  PALETTE.YELLOW, PALETTE.BLACK,  PALETTE.PURPLE, PALETTE.TURQUOISE, PALETTE.GREY,
    PALETTE.RED,    PALETTE.BLUE,   PALETTE.GREEN,  PALETTE.YELLOW, PALETTE.BLACK,  PALETTE.PURPLE, PALETTE.TURQUOISE, PALETTE.GREY])
def closest_point(locations : NDArray, points : NDArray):
    def closest_location_index(location : NDArray):
        displacement = points - location
        distance = np.einsum('ij,ij->i', displacement, displacement)
        return np.argmin(distance)
    closest_indices = []
    for location in locations:
        closest_indices.append(closest_location_index(location))
    return closest_indices
handle_indices_applied = closest_point(handle_locations, V) 
# display handles
grey = np.array([0.99, 0.99, 0.99]) # grey
mesh_color = np.zeros(shape=V.shape)
mesh_color[:, :] = PALETTE.GREY
mesh_color[0] = PALETTE.TURQUOISE
print(f"number of handles: {len(handle_indices_applied)}")
target_pos = np.asarray(target_locations)
handle_pos = V[handle_indices_applied]
arap_before = mp.plot(v=V, f=F, c=mesh_color)
arap_before.add_points(points=handle_pos + np.asarray([0, 0, 0.005]), c=handle_colors,
                          shading={"point_size": 0.1})
arap_before.add_points(points=target_pos + np.asarray([0, 0, -0.10]), c=handle_colors,
                          shading={"point_size": 1})
print(f"big points: handle TARGET positions")
arap_before.add_lines(beginning=handle_pos, ending=target_pos, 
                      shading={"line_color": "blue", "line_width": 1000.0})
arap_before.add_lines(beginning=crestline_V_3d[crestline_E[:,0]], 
                  ending=crestline_V_3d[crestline_E[:,1]], 
                  shading={"line_color": "blue", "line_width": 1000.0})
# The mesh after ARAP
handle_ids_b = np.asarray([[id] for id in handle_indices_applied])
arap_object = igl.ARAP(V, F, 3, handle_ids_b)
V_arap_igl = arap_object.solve(target_pos, V)
arap_result = mp.plot(v=V_arap_igl, f=F, c=PALETTE.GREEN)

In [ ]:
"""SEA algorithm to rubber-sheet crestlines to fit GPR data!"""
TLC_filename = f"{mesh_name}_tlc.txt"
flat_filename = f"{mesh_name}_2d.txt"
write_TLC_input(
    filename=TLC_filename,
    restVert=V,
    initVert=V_arap_igl[:,0:2],
    triangles=F,
    handles=np.asarray(handle_indices_applied))
print(f"the TLC file is: {TLC_filename}")
SEA_input = f"./SEA_QN {TLC_filename} solver_options.txt {flat_filename}"
print(f"in SEA, enter: {SEA_input}")
os.chdir(here)
# in following lines, replace 'where_is_sea' with full path to folder containing SEA code
os.rename(TLC_filename, f"where_is_sea/cmake-build-debug/{TLC_filename}")
os.chdir('where_is_sea/cmake-build-debug')
os.system('pwd')
os.system(SEA_input)
os.rename(flat_filename, f'where_am_i/{flat_filename}')
os.system(f'rm {TLC_filename}')

In [ ]:
"""Visualize result of the SEA algorithm"""
os.chdir(here)
dictionary = read_TLC_result(flat_filename)
V_2d = np.zeros(V.shape)
V_2d[:,0:2] = dictionary["resV"]
flat_plot = mp.plot(v=V_2d, f=F, c=mesh_color)
flat_plot.add_points(
  points=V_2d[handle_indices_applied] + [0, 0, 0.005], c=handle_colors, shading={"point_size": 2})
mesh_lines(vertices=V_2d, faces=F, plot=flat_plot, color="black")

In [ ]:
"""Deform crest lines by proxy of SEA's deformation of the mesh"""
print(crestline_V_3d.shape)
# print(crestline_V_3d[0])
import importlib as imp
import Transfer
imp.reload(Transfer)
mover = Transfer.crestline_mover( 
            mesh_3d_vertices=V, mesh_faces=F, 
            mesh_3d_crestlines_vertices=crestline_V_3d, 
            crestline_edges=crestline_E, 
            mesh_2d_vertices=V_2d, color=mesh_color)
valid_beginning = []
valid_ending = []
tolerance = 1e-3 # if distance to origin is less than this, the point is omitted
for u, v, _ in crestline_E:
    b = mover.mesh_2d_crestlines_vertices[u]
    e = mover.mesh_2d_crestlines_vertices[v]
    if np.dot(b, b) <= tolerance or np.dot(b, b) <= tolerance:
        continue
    else:
        valid_beginning.append(b)
        valid_ending.append(e)
flat_plot_lines = mp.plot(v=V_2d, f=F, c=mesh_color)
flat_plot_lines.add_lines(
                  beginning=np.asarray(valid_beginning), 
                  ending=np.asarray(valid_ending), 
                  shading={"line_color": "blue", "line_width": 1000.0})

In [ ]:
"""save mesh & deformed mesh data"""
np.savetxt(fname=f'{project_name}_mesh_3d_vertices.txt', X=V, fmt='%1.17f')
np.savetxt(fname=f'{project_name}_mesh_2d_vertices.txt', X=V_2d, fmt='%1.17f')
np.savetxt(fname=f'{project_name}_crestline_3d_vertices.txt', X=crestline_V_3d, fmt='%1.17f')
# save crestline 2d vertices as their beginnings and endings
np.savetxt(fname=f'{project_name}_crestline_2d_vertices_start.txt', X=np.asarray(valid_beginning), fmt='%1.17f')
np.savetxt(fname=f'{project_name}_crestline_2d_vertices_end.txt', X=np.asarray(valid_ending), fmt='%1.17f')
np.savetxt(fname=f'{project_name}_crestline_edges.txt', X=crestline_E, fmt='%i')
np.savetxt(fname=f'{project_name}_mesh_faces.txt', X=F, fmt='%i')

In [ ]:
"""Align deformed crest lines to GPR image"""
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl
from PIL import Image

f, ax = plt.subplots(figsize=(32, 32)) # NOT subplot()
# plt.figure(figsize=(36, 36))
ax.set_xlim(0, 300)
ax.set_ylim(0, 300)
init_color = 'red'
warp_color = 'green'
thickness = 2

# imageFile = "Export_230328"
imageFile = '25~89.png'
# img=mpimg.imread(imageFile)
img = Image.open(imageFile)
img = np.flip(img, axis=1)
y_length, x_length, channel = img.shape

imgplot = plt.imshow(img)
tx, ty = [0, 0]
transform = mpl.transforms.Affine2D().translate(-x_length/2, -y_length/2)
transform = transform.rotate(np.deg2rad(180))
transform = transform.scale(0.855) # need conversion
transform = transform.rotate(np.deg2rad(-17.7447))
transform = transform.translate(1470, 1260) # same amount as in blender
imgplot.set_transform(transform + ax.transData)
x = [0, 720, 3311, 2494] # rotating (image border)
y = [1113, 3372, 2546, 0]
plt.plot(x, y, color='black')
"""crestlines (lines)"""
from matplotlib import collections as mc # W method, FAST!
init_lines = [] # item = [(start coord tuple), (end coord tuple)]
warp_lines = []
for i, j, _ in crestline_E:
        x1, y1, _ = crestline_V_3d[i] * 1000
        x2, y2, _ = crestline_V_3d[j] * 1000
        init_lines.append([(x1, y1), (x2, y2)])
        x1, y1, _ = mover.mesh_2d_crestlines_vertices[i] * 1000
        x2, y2, _ = mover.mesh_2d_crestlines_vertices[j] * 1000
        warp_lines.append([(x1, y1), (x2, y2)])
init_line_collection = mc.LineCollection(init_lines, colors=init_color, linewidths=thickness, linestyle='solid')
plt.plot(crestline_V_3d[:,0] * 1000, crestline_V_3d[:,1] * 1000, 
         'o', markersize=thickness, color=init_color)
warp_line_collection = mc.LineCollection(warp_lines, colors=warp_color, linewidths=thickness, linestyle='solid')
plt.plot(mover.mesh_2d_crestlines_vertices[:,0] * 1000, mover.mesh_2d_crestlines_vertices[:,1] * 1000, 
         'o', markersize=thickness, color=warp_color)

COURIER = {'fontname':'Courier'}
ax.text(0.01, 0.95, 'Original crestlines',
        verticalalignment='bottom', horizontalalignment='left',
        transform=ax.transAxes,
        color=init_color, fontsize=24, **COURIER)
ax.text(0.01, 0.90, 'WARPed crestlines',
        verticalalignment='bottom', horizontalalignment='left',
        transform=ax.transAxes,
        color=warp_color, fontsize=24, **COURIER)
ax.text(0.99, 0.95, 'Point = handle  ',
        verticalalignment='bottom', horizontalalignment='right',
        transform=ax.transAxes,
        color='black', fontsize=24, **COURIER)
ax.text(0.99, 0.90, 'Line = crestline',
        verticalalignment='bottom', horizontalalignment='right',
        transform=ax.transAxes,
        color='black', fontsize=24, **COURIER)

ax.autoscale()
current_figure = plt.gcf()
plt.show()
plt.draw()
current_figure.savefig('PLOT.png', dpi=96, bbox_inches='tight')

In [ ]:
"""Align deformed crest lines with GPR image, then visualize"""
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl
from PIL import Image

f, ax = plt.subplots(figsize=(32, 32)) # NOT subplot()
# plt.figure(figsize=(36, 36))
ax.set_xlim(0, 300)
ax.set_ylim(0, 300)
init_color = 'red'
warp_color = 'green'
thickness = 2
imageFile = 'gpr_image.png'
img = Image.open(imageFile)
img = np.flip(img, axis=1)
y_length, x_length, channel = img.shape

imgplot = plt.imshow(img)
tx, ty = [0, 0]
transform = mpl.transforms.Affine2D().translate(-x_length/2, -y_length/2)
transform = transform.rotate(np.deg2rad(180))
transform = transform.scale(0.855) # need conversion
transform = transform.rotate(np.deg2rad(-17.7447))
transform = transform.translate(1470, 1260)
imgplot.set_transform(transform + ax.transData)
x = [0, 720, 3311, 2494]
y = [1113, 3372, 2546, 0]
plt.plot(x, y, color='black')
"""crestlines (lines)"""
from matplotlib import collections as mc # W method, FAST!
init_lines = [] # item = [(start coord tuple), (end coord tuple)]
warp_lines = []
for i, j, _ in crestline_E:
        x1, y1, _ = crestline_V_3d[i] * 1000
        x2, y2, _ = crestline_V_3d[j] * 1000
        init_lines.append([(x1, y1), (x2, y2)])
        x1, y1, _ = mover.mesh_2d_crestlines_vertices[i] * 1000
        x2, y2, _ = mover.mesh_2d_crestlines_vertices[j] * 1000
        warp_lines.append([(x1, y1), (x2, y2)])
init_line_collection = mc.LineCollection(init_lines, colors=init_color, linewidths=thickness, linestyle='solid')
plt.plot(crestline_V_3d[:,0] * 1000, crestline_V_3d[:,1] * 1000, 
         'o', markersize=thickness, color=init_color)
warp_line_collection = mc.LineCollection(warp_lines, colors=warp_color, linewidths=thickness, linestyle='solid')
plt.plot(mover.mesh_2d_crestlines_vertices[:,0] * 1000, mover.mesh_2d_crestlines_vertices[:,1] * 1000, 
         'o', markersize=thickness, color=warp_color)

COURIER = {'fontname':'Courier'}
ax.text(0.01, 0.95, 'Original crestlines',
        verticalalignment='bottom', horizontalalignment='left',
        transform=ax.transAxes,
        color=init_color, fontsize=24, **COURIER)
ax.text(0.01, 0.90, 'WARPed crestlines',
        verticalalignment='bottom', horizontalalignment='left',
        transform=ax.transAxes,
        color=warp_color, fontsize=24, **COURIER)
ax.text(0.99, 0.95, 'Point = handle  ',
        verticalalignment='bottom', horizontalalignment='right',
        transform=ax.transAxes,
        color='black', fontsize=24, **COURIER)
ax.text(0.99, 0.90, 'Line = crestline',
        verticalalignment='bottom', horizontalalignment='right',
        transform=ax.transAxes,
        color='black', fontsize=24, **COURIER)

ax.autoscale()
current_figure = plt.gcf()
plt.show()
plt.draw()
current_figure.savefig('PLOT.png', dpi=96, bbox_inches='tight')

In [ ]:
"""Get GPR centerlines from GPR image at different depths"""
files = [] # list of filenames of GPR images at different depths
result_dictionary = {}

In [ ]:
from PIL import Image, ImageOps
from PIL import ImageOps
from Display import Plot_2D
from DualColor import binary

In [ ]:
"""Mask for deformed ARAP mesh"""
crestline_area_mask_image_filename = f'{project_name}_crestlines_mask.png'
if os.path.isfile(crestline_area_mask_image_filename):
    print(f"loading existing image")
    crestlines_mask_image = Image.open(crestline_area_mask_image_filename)
    crestlines_mask, _ = binary(np.asarray(ImageOps.grayscale(crestlines_mask_image)), 0, [255, 255, 255], [0, 0, 0])
else:
    print(f"making new image")
    # in below line, replace 'gpr_image' with filename of GPR image
    img_display = Image.open('gpr_image.png')
    img_display = np.flip(img_display, axis=0)
    y_length, x_length = img_display.shape
    thic = 1.6
    thin = 0.1
    plotter = Plot_2D(init_color='black', warp_color='white', thickness=thin)
    f, ax = plt.subplots(figsize=(32, 32)) # NOT subplot()
    print(f"image is {x_length}, {y_length}")
    ax.set_xlim(0, x_length)
    ax.set_ylim(0, y_length)
    plotter.Plot_Background(f, ax, img_display, "BLACK")
    for face in F:
        v1, v2, v3 = face
        triangle = plt.Polygon((V_2d[v1][0:2] * 1000, 
                                V_2d[v2][0:2] * 1000, 
                                V_2d[v3][0:2] * 1000), 
            facecolor='white', edgecolor='white')
        ax.add_patch(p=triangle)
    ax.autoscale()
    current_figure = plt.gcf()
    plt.show()
    plt.draw()
    """save part of figure"""
    DPI = 169.18 # manually tuned
    plotter.Plot_SaveTrim(f, ax, current_figure, crestline_area_mask_image_filename, 
                        (0, 0), (x_length, x_length)) # save all
crestlines_mask_image = Image.open(crestline_area_mask_image_filename)
crestlines_mask, _ = binary(np.asarray(ImageOps.grayscale(crestlines_mask_image)), 0, [255, 255, 255], [0, 0, 0])
np.savetxt(fname=f'{project_name}_crestlines_mask.txt', X=crestlines_mask, fmt='%i')

In [ ]:
"""Mask for GPR image"""
crestline_area_mask_image_filename = f'{project_name}_radar_mask.png'
if os.path.isfile(crestline_area_mask_image_filename):
    print("reading from existing image")
else:
    print(f"making new image")
    # in below line, replace 'gpr_image' with filename of GPR image
    radar_image = Image.open('gpr_image.png')
    img_display = np.flip(radar_image, axis=0)
    y_length, x_length, _ = img_display.shape
    radar_mask_upright = np.zeros(shape=(y_length, x_length))
    for x in range(x_length):
        for y in range(y_length):
            if img_display[y][x][3] == 0:
                radar_mask_upright[y][x] = 1
    thic = 1.6
    thin = 0.1
    plotter = Plot_2D(init_color='black', warp_color='white', thickness=thin)
    f, ax = plt.subplots(figsize=(32, 32)) # NOT subplot()
    print(f"image is {x_length}, {y_length}")
    ax.set_xlim(0, x_length)
    ax.set_ylim(0, y_length)
    plotter.Plot_Background(f, ax, img_display, "BLACK")
    plotter.Plot_ImageTransform(f, ax, 1 - (radar_mask_upright) * 255)
    ax.autoscale()
    current_figure = plt.gcf()
    plt.show()
    plt.draw()
    """save part of figure"""
    DPI = 169.18 # manually tuned
    plotter.Plot_SaveTrim(f, ax, current_figure, crestline_area_mask_image_filename, 
                        (0, 0), (x_length, x_length)) # save all
# get GPR image area mask & union mask
radar_mask_image = Image.open(crestline_area_mask_image_filename)
radar_mask, _ = binary(np.asarray(ImageOps.grayscale(radar_mask_image)), 0, [255, 255, 255], [0, 0, 0])
np.savetxt(fname=f'{project_name}_radar_mask.txt', X=radar_mask, fmt='%i')

In [ ]:
"""Calculate the area in which validation is done
This is the intersection of the above two masks"""
union_mask = crestlines_mask * radar_mask
union_mask = np.asarray(union_mask)
np.savetxt(fname=f'{project_name}_union_mask.txt', X=union_mask, fmt='%i')

In [ ]:
"""Rasterize crest lines
Note that GPR images do not need to be processed this way, as they come out of
the ridge detection algorithm rasterized"""
import Display
imp.reload(Display)
crestlines_lines_mask_filename = f'{project_name}_crestlines_lines.png'
if os.path.isfile(crestlines_lines_mask_filename):
    print("use existing image")
else:
    print("making new image")
    radar_image = Image.open('gpr_image.png')
    img_display = np.flip(radar_image, axis=0)
    y_length, x_length, _ = img_display.shape
    plotter = Display.Plot_2D(init_color='black', warp_color='white', thickness=0.1)
    f, ax = plt.subplots(figsize=(32, 32)) # NOT subplot()
    ax.set_xlim(0, x_length)
    ax.set_ylim(0, y_length)
    plotter.Plot_Background(f, ax, img_display, "black")
    plotter.Plot_Crestlines_Warp(f, ax, begin=valid_beginning, end=valid_ending)
    ax.autoscale()
    current_figure = plt.gcf()
    plt.show()
    plt.draw()
    plotter.Plot_SaveTrim(f, ax, current_figure, crestlines_lines_mask_filename, 
                        (0, 0), (x_length, x_length)) # save all
crestlines_lines_mask_image = Image.open(crestlines_lines_mask_filename)
crestlines_lines_mask, _ = binary(np.asarray(ImageOps.grayscale(crestlines_lines_mask_image)), 0, [255, 255, 255], [0, 0, 0])
np.savetxt(fname=f'{project_name}_crestlines_lines_mask.txt', X=crestlines_lines_mask, fmt='%i')

In [ ]:
"""Exporting crest lines as .ply files"""
os.chdir(here)
def export_lines(vertices, edges, filename):
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(vertices),
        lines=o3d.utility.Vector2iVector(edges),
    )
    o3d.io.write_line_set(filename, line_set, print_progress=True)

export_lines(vertices=crestline_V_3d,
             edges=crestline_E[:,0:2].astype(int),
             filename=f"{mesh_name}_n={neighbor}_3d_lines.ply")
print(f"saved mesh file: {mesh_name}_n={neighbor}_3d_lines.ply")